# Chapter 9: Backpropagation

In [11]:
# Preface: Install necessary packages:
import numpy as np
import matplotlib.pyplot as plt
import math
import nnfs
from nnfs.datasets import spiral_data
from timeit import timeit
from resources.classes import DenseLayer, ReLU, SoftMax, Loss, CategoricalCrossEntropy, SoftMaxCategoricalCrossEntropy

## Section 1: Backprop. Intro

We'll start off the chapter by backpropagating the ReLU function for a single neuron with the goal of minimizing **the output** from this neuron. This won't directly translate to our model ops, since the goal there is minimize **loss**, but it does serve as a good example showing how the process would work.

Let's initialize a neuron:


In [ ]:
# Creating input list of length 3
x = [1.0, -2.0, 3.0]
# Creating random weights
w = [-3.0, -1.0, 2.0]
# Setting bias variable
b = 1

xw0 = x[0] * w[0]
xw1 = x[1] * w[1]
xw2 = x[2] * w[2]
z = xw0 + xw1 + xw2 + b
# This could have been done just using a "z = np.dot(x, w) + b", but the format we've chosen is more convenient for our experimentation
print(f"the layer output before act. function is {z}")

y = max(z, 0)
print(f"the neuron output is {y}")

Now that was a full forward pass through the (made up) data! Now we can think about how to approach backpropagation.

First, lets imagine what our function is actually doing, which can be roughly interpreted as $ReLU(\sum[inputs * weights] + bias)$ and which we can write more specifically as $ReLU(x0w0 + x1w1 + x2w2 + bias)$. We will rewrite this as $y = ReLU(sum(mul(x0, w0), mul(x1, w1), mul(x2, w2), bias))$ for the purposes of easier derivation. If we're trying to find the derivative of y with respect to x0, we can write the following:
$$
\frac{\partial}{\partial x_{0}}[ReLU(sum(mul(x0, w0), mul(x1, w1), mul(x2, w2), bias))] = \\
\frac{dReLU()}{dSum()} \cdot \frac{\partial sum()}{\partial mul(x_{0}, w_{0})} \cdot \frac{\partial mul(x_{0}), w_{0}}{\partial x_{0}}
$$
Now, if we were to just solve this out, we would see the impact that $x_{0}$ is actually having on the output.

During the backward pass, what we actually do is calculate the derivative of the loss function and multiply it with the derivative of the activation function, and then the derivative of the output layer, and so on, all the way through the hidden layers and activation functions.

In all of these layers, the derivative with respect to the weights and biases will form the gradients that will tell us how to update our weights and biases.

Let's work backwards through our network now, assuming that the neuron receives a gradient of 1 from the next layer.

The first step in our process is calculating the derivative of the ReLU activation function -- which we've already done before! I'll write it out below: 
$$
f(x) = max(x, 0) \rightarrow \frac{d}{dx} f(x) = 1(x > 0)
$$

Now, lets move to using this in python.

In [ ]:
# Make sure you have run the previous code cell so there is a z to go off.

# Hard-coding the gradient from the previous layer
dValue = 1.0

# The RHS of the below is the derivative of the ReLU function with respect to z, because z denotes the neuron's output. 
dReluDz = dValue * (1. if (z > 0) else 0.)
print(dReluDz)

Now with our ReLU derivative handled, the immediately preceding operation was the summation of the weights inputs and bias. So, here we need to calculate a partial derivative of the sum function and then use the chain rule to multiply it by the derivative of the outer function -- which is the ReLU.  

We can begin defining the partial derivatives:
- dReluDxw0 -- the partial derivative of RELU w.r.t. the first weighted input, x0w0
- dReluDxw1 -- the partial derivative of RELU w.r.t. the second weighted input, x1w1
- dReluDxw2 -- the partial derivative of RELU w.r.t. the third weighted input, x2w2
- dReluDb -- the partial derivative of RELU w.r.t. the bias, b

As we know, the partial derivative of any sum operation is always 1, no matter what the inputs are.

So, we can now incorporate this into our python.

In [ ]:
# Make sure you have run the previous code cells so there is a dReluDz to go off.

# I'm just going to make one variable, since all of it will just be 1
dSumDxwX = 1
dSumDb = 1

# Now let's calculate the derivative for each
dReluDxw0 = dReluDz * dSumDxwX
dReluDxw1 = dReluDz * dSumDxwX
dReluDxw2 = dReluDz * dSumDxwX
dReluDb = dReluDz * dSumDb

print(dReluDxw0, dReluDxw1, dReluDxw2, dReluDb)

Great, so that's the summation function! Now, we have to do arguably the most complex one: the multiplication function.

As we can remember, the derivative for a product is whatever the input is being multiplied by, as I'll show below:
$$
f(x,y) = x \cdot y \rightarrow \frac{\partial}{\partial x} f(x,y) = y \\
\frac{\partial}{\partial y} f(x,y) = x \\
$$

Following this, the partial derivative of the first weighted input $(x \cdot w)$ with respect to the input (x) is just the weight (w) -- as it is the other input of the function.

So, let's add this functionality to our code.

In [ ]:
# Pull the variables
dMulDx0 = w[0]
dMulDx1 = w[1]
dMulDx2 = w[2]
dMulDw0 = x[0]
dMulDw1 = x[1]
dMulDw2 = x[2]

# Actually calculate the derivative
dReluDx0 = dReluDxw0 * dMulDx0
dReluDx1 = dReluDxw1 * dMulDx1
dReluDx2 = dReluDxw2 * dMulDx2
dReluDw0 = dReluDxw0 * dMulDw0
dReluDw1 = dReluDxw1 * dMulDw1
dReluDw2 = dReluDxw2 * dMulDw2

print(dReluDx0, dReluDw0, dReluDx1, dReluDw1, dReluDx2, dReluDw2)

Now that is our entire set of neuronal partial derivatives with respect to the inputs, weights, and the bias. We can now use this to optimize these calculations. 

All together, these can be represented as:

In [ ]:
dx = [dReluDx0, dReluDx1, dReluDx2] # the gradients on inputs
dw = [dReluDw0, dReluDw1, dReluDw2] # the gradients on the weights
db = dReluDb # the gradient on the bias, of which there is just one

print(dx, dw, db)

We'll now use these to see how we can change our weights to minimize the output (as was our goal for this example), but we would normally use them in our optimizer to improve the output.  

If we take a look at our current weights, bias, and output:

In [ ]:
print(f"{w}, {b}, {z}")

Now, we can use our calculated partial derivatives to play with this and see if we can decrease output:

In [ ]:
w[0] += -0.001 * dw[0]
w[1] += -0.001 * dw[1]
w[2] += -0.001 * dw[2]
b += -0.001 * db

print(w, b)

Lets perform a forward pass to see how this impacts our final output:

In [ ]:
# Multiply inputs and weights
xw0 = x[0] * w[0]
xw1 = x[1] * w[1]
xw2 = x[2] * w[2]

# Add up mult + bias
z = xw0 + xw1 + xw2 + b

# ReLU function for output
y = max(z, 0)

print(y)

That means that we've reduced our output! While it's only by a very tiny bit, 6.0 vs 5.985, it shows us that we're trending in the right direction! Like I said, optimizing a single neuron for the pure sake of minimizing it's output is something that won't translate into the real world, but it's a step. What we're actually going to be doing is working to decrease the final loss value 

Our next objective will be to apply this to a list of samples and expand it to a whole layer of neurons. In this example, our neural net will consist of a single hidden layer with 3 neurons (each with 3 inputs and 3 weights). Let's set up below:

In [ ]:
# We'll make up the gradients from the "next" layer for the sake of this example
dvalues = np.array([[1.0, 1.0, 1.0]])

# We have 3 sets of weights and 4 inputs, meaning we need 4 weights each.
weights = np.array([[0.2, 0.8, -0.5, 1],
                   [0.5, -0.91, 0.26, -0.5],
                   [-0.26, -0.27, 0.17, 0.87]]).T

# Sum the weights of inputs and multipy by the gradients
dx0 = sum(weights[0]*dvalues[0])
dx1 = sum(weights[1]*dvalues[0])
dx2 = sum(weights[2]*dvalues[0])
dx3 = sum(weights[3]*dvalues[0])

dInputs = np.array([dx0, dx1, dx2, dx3])

print(dInputs)

From this, we see how dInputs is the gradient of the neuron function with respect to the outputs.

However, we can simplify this tremendously by just using np.dot!  

In [ ]:
dInputs = np.dot(dvalues[0], weights.T)
print(dInputs)

That about does it -- but we're missing one thing: the ability to handle samples in our batch. Let's implement that now:

In [ ]:
# We'll create gradient values for each batch
dvalues = np.array([[1.0, 1.0, 1.0],
                    [2.0, 2.0, 2.0],
                    [3.0, 3.0, 3.0]])

dInputs = np.dot(dvalues, weights.T)

print(dInputs)

Those are our gradients with respect to the inputs. That was a lot. So, now we should take a look at our gradients with respect to the weights. 

In [ ]:
# We have 3 sets of sample inputs
inputs = np.array([[1, 2, 3, 2.5],
                   [2, 5, -1, 2],
                   [-1.5, 2.7, 3.3, -0.8]])

# Notice how this time we flip the position of inputs.T and dvalues so that the arrangement is (n x m) and (m x p).
dweights = np.dot(inputs.T, dvalues)

print(dweights)

This correspondingly matches our shape of weights because we've summed the inputs for each weight and then multipled it by the input gradient. We can do this for biases as well!

In [ ]:
# One bias for each neuron
biases = np.array([[2, 3, 0.5]])

# Sum it over the samples and keep the row vector dimensions
dbiases = np.sum(dvalues, axis=0, keepdims=True)

print(dbiases)

Finally, we should also account for the ReLU function, which is 1 when > 0, 0 otherwise.

In [ ]:
# Creating a random array of layer outputs
z = np.array([[1, 2, -3, -4],
              [2, -7, -1, 3],
              [-1, 2, 5, -1]])

dvalues = np.array([[1, 2, 3, 4],
                    [5, 6, 7, 8],
                    [9, 10, 11, 12]])

# np.zeros_like(arg) is a function that returns an array of the same size as the arg but filled with 0's
drelu = np.zeros_like(z)
# This iterates through the elements and if z > 0, sets it to 1.
drelu[z > 0] = 1
print(drelu)

# Apply the chain rule
drelu *= dvalues
print(drelu)

I'm going to update our classes to account for what we've learned so far in this chapter, but I'm going to detail everything to check out from those changes:
- Within the DenseLayer class:
    - Added "self.inputs" as a object in the forward method
    - Created the "backward" method and its corresponding process
- Within the ReLU class:
    - Added "self.inputs" as a object in the forward method
    - Created the "backward" method and its corresponding process

## Section 2: Categorical Cross-Entropy Loss Derivatives

As seen in chapter five, the equation for the categorical cross-entropy loss function is:
$$
L_{i} = -log(\hat{y}_{i,k})
$$
This is convenient for calculations, but for the sake of our process of finding the derivative, we will us the full equation:
$$
L_{i} = \sum_{j} y_{i, j}log(\hat{y}_{i,j})
$$
The reason we use the latter is because the goal of finding the gradient means that we need the partial derivatives of the loss function with respect to **each** of it's inputs, meaning we can't use the shorted version.

Let us define the full equation as being:
$$
\frac{\partial L_{i}}{\partial \hat{y}_{i,j}}
= 
\frac{\partial}{\partial \hat{y}_{i,j}}
[-\sum_{j} y_{i,j}log(\hat{y}_{i,j})]
$$

As we know that we can remove constants, let us rewrite the equation as follows:
$$
-\sum_{j} y_{i,j} \cdot \frac{\partial}{\partial \hat{y}_{i,j}} log(\hat{y_{i,j}})
$$

I'm going to skip re-writing some steps, but this finally solves to
$$
 -\frac{y_{i,j}}{\hat{y}_{i,j}}
$$

Great - now we know what the derivative looks like and why! I'm updating the classes.py file, and I've modified the following things:
- Created the "backward" method in the categorical cross entropy loss class. It basically works by first turning numerical labels into one-hot encoded vectors; then performs gradient normalization to prevent us from having to change the learning rate for each sample. 

## Section 3: Softmax Activation Derivative 

Let's write out the formula for the softmax equation to stir our brains:
$$
S_{i,j} = \frac{e^{z_{i,j}}}{\sum_{l=1}^{L} e^{z_{i,l}}}
\rightarrow
\frac{\partial S_{i,j}}{\partial z_{i,k}}
=
\frac{\partial \frac{e^{z_{i,j}}}{\sum_{l=1}^{L} e^{z_{i,l}}}}{\partial z_{i,k}}
$$
Where $S_{i,j}$ denotes the j-th Softmax's output of the i-th sample, z -- input array which is a list of input vectors, $z_{i,j}$ -- j'th Softmax's input of i'th sample, L -- number of inputs, $z_{i,k}$ -- k'th Sotmax's input of i'th sample.

Here, we're basically calculating the Jacobian matrix of the vectors, as we're trying to find the partial derivative of every output with respect to each input.

Let's remember the quotient rule:
$$
f(x) = \frac{g(x)}{h(x)} \rightarrow f'(x) = \frac{g'(x) \times h(x) - g(x) \times h'(x)}{[h(x)]^{2}}
$$ 

For my sake and yours, I'm going to skip a lot of the math here, but the precursor you need to know is the Kronecker delta function:
$$
\delta_{i,j} = \begin{cases}
1 \text{ if } i = j \\
0 \text{ if } i \neq j \\
\end{cases}
$$
We can then use this to furthest simplify our equation to:
$$
\frac{\partial S_{i,j}}{\partial z_{i,k}} = S_{i,j} \cdot (\delta_{i,j} - S_{i,k})
$$
For ease of implementation in python, we can re-write this as:
$$
\frac{\partial S_{i,j}}{\partial z_{i,k}}
= 
S_{i,j}\delta_{j,k} - S_{i,j}S_{i,k}
$$

The above lets us do this in just two numpy functions! Let me show below.

In [ ]:
# Creating a random softmax output
example_softmax = np.array([0.7, 0.1, 0.2]).reshape(-1,1)

# Creates a generic I matrix with 1's along the diagonal and 0's elsewhere
identity = np.eye(example_softmax.shape[0])

# Multiply... self explanatory.
print(f"Method one: \n{identity * example_softmax}")

# BUT: This can be simplified into just one np.diagflat() call!

print(f"Method two: \n{np.diagflat(example_softmax)}")

Now, the above takes care of the first $S_{i,j}\delta_{j,k}$ part, but we're still left with the $S_{i,j}S_{i,k}$ part. That means, we just have to do x dot x.T, which I'll show below: 

In [ ]:
print(np.dot(example_softmax, example_softmax.T))

Finally, we can go and perform subtraction of both arrays:

In [ ]:
print(np.diagflat(example_softmax) - np.dot(example_softmax, example_softmax.T))

The above matrix is our Jacobian matrix -- which is our array of partial derivatives of in all combinations of both input vectors. We calculate the partial derivative of every output of the Softmax function with respect to each input separately because each input influences each output because of the normalization process.  

This results in a list of Jacobian matrices which effectively forms a 3D matrix. That can be visualized as a column whose levels are Jacobian matrices being the sample-wise gradient of the Softmax function.

I'm going to go and add some more stuff to our classes.py, which I'll explain here:
- Added the "backward" method of the SoftMax class, which works by: 1. creating an empty array with the same shape as the gradients to be prepared for our application of the chain rule; 2. iterating sample-wise over the pairs of outputs and gradients, calculating the partial derivatives, final product, and gradient vector in the process, storing them in each row as we go. 

Interesting side note: **every** gradient is a Jacobian, but **not every** Jacobian is a gradient.

## Section 4: Common Categorical Cross Entropy Loss and Softmax Activation Derivative

We can combine the previous two sections to present a unified derivative: the derivative of the loss function with respect to the softmax inputs. We can define by applying the chain rule:
$$
\frac{\partial L_{i}}{\partial z_{i,k}}
=
\frac{\partial L_{i}}{\partial \hat{Y}_{i,j}} \cdot \frac{\partial S_{i,j}}{\partial z_{i,k}}
$$

This partial derivative formula is just the partial derivative of the loss function with respect to its inputs, multiplied (using the chain rule) times the partial derivative of the activation function with respect its inputs.

I'll skip ALL the calculus because it's not necessary for the purposes of these public notes, but I would recommend going to the book and reading through the section.

In the end, this simplifies down to:
$$
\hat{y}_{i,k} - y_{i,k} 
$$ 

This is drastically simpler and more efficient than the two discrete partial derivatives we were using before. As such, I'm going to create or update our classes in the class.py file as such:
- Created new "SoftMaxCategoricalCrossentropy" class which combines both of the SoftMax and CategoricalCrossEntropy classes.
- Implemented the forward and backward passes of that class, which make use of the methods from the extended classes for efficiency. 

Our solution is so much more efficient because it takes advantage of y_true being one-hot encoded, which means that, for each sample, there is only a singular 1 in those vectors and the remaining positions are filled with 0's. 

Finally! We can test if our combined backward step returns the same values compared to when we backpropagate gradients through both of the functions separately. For this test, we'll use random values.

In [10]:
# Initializing random data generator
nnfs.init()

# Some random softmax outputs, each row adding up to 1
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])

# Creating ground truth labels
class_targets = np.array([0, 1, 1])

# Our combined method
def f1():
    softmaxLoss = SoftMaxCategoricalCrossEntropy()
    softmaxLoss.backward(softmax_outputs, class_targets)
    dvalues1 = softmaxLoss.dinputs
    return dvalues1

# Our separate method
def f2():
    activation = SoftMax()
    activation.output = softmax_outputs
    loss = CategoricalCrossEntropy()
    loss.backward(softmax_outputs, class_targets)
    activation.backward(loss.dinputs)
    dvalues2 = activation.dinputs
    return dvalues2
    
dvalues1 = f1()
dvalues2 = f2()

t1 = timeit(lambda: f1(), number=10000)
t2 = timeit(lambda: f2(), number=10000)

print(f"Gradients: combined loss and activation: {dvalues1}")
print(f"Gradients: separate loss and activation: {dvalues2}")

print(f"F2 takes {t2/t1} times more than F1!!!")


Gradients: combined loss and activation: [[-0.1         0.03333333  0.06666667]
 [ 0.03333333 -0.16666667  0.13333333]
 [ 0.00666667 -0.03333333  0.02666667]]
Gradients: separate loss and activation: [[-0.09999999  0.03333334  0.06666667]
 [ 0.03333334 -0.16666667  0.13333334]
 [ 0.00666667 -0.03333333  0.02666667]]
F2 takes 17.969141402203054 times more than F1!!!


As we can see, aside from an ultra-minor rounding difference (resulting from the precision of floating points in raw Python vs NumPy) these are the same outputs!! However, as we see on the outputs, F2 takes significantly longer than F1, due to it's lack of efficiency optimizations.

With the above in mind, we can finally code up a final version of the model in this chapter!

In [13]:
# Initialize data
X, y = spiral_data(samples=100, classes=3)

# Create dense layer with 2 input features and 3 output values
dense1 = DenseLayer(2, 3)

# Add a ReLU activation function to be used on this
activation1 = ReLU()

# Create a 2nd dense layer with 3 input features and 3 output values
dense2 = DenseLayer(3, 3)

# Create a softmax classifier for the output
activation2 = SoftMaxCategoricalCrossEntropy()

# Forward pass through first layer
dense1.forward(X)

# Apply ReLU to output of Dense1
activation1.forward(dense1.output)

# Forward pass through second layer
dense2.forward(activation1.output)

# Calculate loss
loss = activation2.forward(dense2.output, y)

print(activation2.output[:5])
print(f"Loss: {loss}")

# Calculate accuracy
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

print(f"Accuracy: {accuracy}")

# Backward pass
activation2.backward(activation2.output, y)
dense2.backward(activation2.dinputs)
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

# Printing gradients
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)

[[0.33333334 0.33333334 0.33333334]
 [0.33333355 0.3333332  0.3333332 ]
 [0.33333382 0.33333313 0.3333331 ]
 [0.3333341  0.33333302 0.33333296]
 [0.33333433 0.3333329  0.33333278]]
Loss: 1.098608136177063
Accuracy: 0.33666666666666667
[[ 3.3042468e-06 -3.9488241e-06 -9.9410368e-05]
 [-2.2006872e-05  3.0671345e-04  1.6974623e-04]]
[[-1.8163288e-05 -5.1999162e-04  1.4667885e-05]]
[[ 9.1446236e-05 -2.5220116e-04  1.6075492e-04]
 [-1.7278348e-04  3.9700870e-04 -2.2422522e-04]
 [ 4.4883702e-05 -1.2783038e-04  8.2946674e-05]]
[[ 4.6612695e-06 -8.3957566e-06  3.5986304e-06]]


The final model compilation really helped solidify everything into place for me. We can now actually track model performance and almost use it to tune our parameters in hope of improving it -- and that's what we'll cover in the next chapter of the book: optimizers!

### Anyways, that's it for this chapter! Thanks for following along with my annotations of *Neural Networks from Scratch* by Kinsley and Kukieła!